Tensorflow is a powerful library for numerical computation, particularly well suited and fine-tuned for large-scale ML.

Its core is very similar to NumPy, but with GPU support.

It also supports distributed computing. 

It includes a kind of just-in-time (JIT) compiler that allows it to optimize computations for speed and memory usage.

Computations graphs can be exported to a portable format, so we can train a Tensorflow model in one environment and run it in another.

It offers many more features besides tf.keras, i.e. it has data loading and preprocessing ops (tf.data, tf.io, etc.), image processing ops (tf.image), signal processing ops (tf.signal), and more.


### Using Tensorflow like numpy

In [1]:
import tensorflow as tf

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

In [5]:
print(t)
print(t.shape)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
(2, 3)


In [6]:
t.dtype

tf.float32

In [7]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
t * 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[10., 20., 30.],
       [40., 50., 60.]], dtype=float32)>

In [11]:
t@tf.transpose(t) # Here @ is used for matrix multiplication

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [13]:
tf.transpose(t)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 4.],
       [2., 5.],
       [3., 6.]], dtype=float32)>

#### Keras' Low-Level API

The Keras API actually has its own low-level API, located in keras.backend. In keras.backend, the functions generally just call the corresponding Tensorflow operations. 

In [14]:
from tensorflow import keras

K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### Tensors and NumPy

We can create a tensor from a NumPy array, and vice versa, and we can even apply Tensorflow operations to NumPy arrays and NumPy operations to tensors.

In [15]:
import numpy as np

a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [18]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

#### Type Conversions
Type conversions can significantly hurt performance. Tensorflow does not perform any type conversions automatically: it just raises an exception if we try to execute an operation on tensors with incompatible types.

In [19]:
tf.constant(2.) + tf.constant(10)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [20]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables

As constant tensors can not be modified, so there's variable tensors.

In [3]:
import tensorflow as tf

v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])

In [6]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 8., 16., 24.],
       [32., 40., 48.]], dtype=float32)>

In [8]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 8., 42., 24.],
       [32., 40., 48.]], dtype=float32)>

In [14]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 8., 42.,  0.],
       [32., 40.,  1.]], dtype=float32)>

In [15]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [ 32.,  40., 200.]], dtype=float32)>

### Customizing Models and Training Algorithms:

Customize loss fuction

In [37]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) /2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [42]:
# Creating a subclass of the keras.losses.Loss class.

from tensorflow import keras

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < 1
        squared_loss = tf.square(error) /2
        linear_loss = tf.abs(error) - 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

### Custom Activation Functions, Initializers, Regularizers, and Constraints

In [3]:
import tensorflow as tf

In [2]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

In [4]:
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

In [5]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))

In [6]:
def my_positive_weights(weights):
    return tf.where(weights<0., tf.zeros_like(wieghts), weights)

In [11]:
from tensorflow import keras

class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

#### Streaming Metrics

Here, we created a Precision object, then we used it like a function, passing it the labels and predictions for the first batch, then for the second batch we passed the labels and predictions.

After the first batch it returns the precision of 80%, then after the second batch it returns 55.56% which is the overall precision so far, not the second batch's precision. This is called a streaming metric or stateful metric.

We can reset these variables using the `reset_states()` method.

In [5]:
from tensorflow import keras

precision = keras.metrics.Precision()
print(precision([0, 1, 1, 1, 0, 0, 1], [1, 1, 1, 1, 0, 0, 1]))

tf.Tensor(0.8, shape=(), dtype=float32)


In [7]:
print(precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1,1 ,0, 0, 0, 1]))

tf.Tensor(0.5555556, shape=(), dtype=float32)


In [8]:
 precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5555556>

In [9]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([5.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [11]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [12]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    
    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

### Custom Layers

In [1]:
import tensorflow as tf
from tensorflow import keras

exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [5]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name="kernel", shape=[batch_input_shape[-1], self.units], initializer='glorot uniform')
        self.bias = self.add_weight(name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)
        
    def call(self, x):
        return self.activation(X @ self.kernel + self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units, 'activation': keras.activations.serialize(self.activation)}

In [8]:
# If we want to add Guassian noise for regularization for training.
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape